# Overview

Here, we will demonstrate how to carry out geometry optimizations using ASE. For this, we need to pick an energy model, which in ASE is called a `Calculator`. The `Calculator` is the simulation engine that, given an atomic configuration, will return the energy of the system (as well as forces and, for solids, the unit cell stresses).

For this exercise, we will mainly use a "machine learning potential". We will cover machine learning potentials in greater detail later in the course, but it is a machine learning model trained to reproduce density functional theory energy, forces, and stresses at a low computational cost. The model we will use in this exercise is called TensorNet-MatPES-r2SCAN as reported in https://arxiv.org/abs/2503.04070.

For additional details on how to carry out geometry optimizations in ASE, refer to the [Structure Optimization](https://ase-lib.org/ase/optimize.html) section of the documentation.


# Setup


In [ ]:
%pip install "ase@git+https://gitlab.com/ase/ase.git" "matcalc[matgl]>=0.4.5"

In [ ]:
!curl "https://github.com/Andrew-S-Rosen/cbe423/blob/main/lecture5/mof5.cif" -o mof5.cif

# Demonstration


## Preparing the `Atoms` object and `Calculator`


First, we will need to define our atomic system. We will study a porous material called a metal-organic framework, specifically MOF-5.


We can start by reading in our crystal structure with ASE and visualizing it.


In [ ]:
from ase.io import read
from ase.visualize import view

original_atoms = read("mof5.cif")
view(original_atoms)

Since we want our calculations to be efficient, let's instead start by making a primitive cell using Pymatgen, which we will then convert to an ASE `Atoms` object.


In [ ]:
from pymatgen.core import Structure

structure = Structure.from_file("mof5.cif", primitive=True)
atoms = structure.to_ase_atoms()

Now let's view the new `Atoms` object. You'll see that it is a primitive cell and contains fewer atoms. If you want to make it a bit more fun, go ahead and distort one of the atoms a little bit in the GUI.


In [ ]:
from ase.visualize import view

view(atoms)

Now we need to define a calculator. ASE comes with many pre-made calculators (https://ase-lib.org/ase/calculators/calculators.html), and there are many external codes that ship their own ASE calculators.


We will load the TensorNet-MatPES-r2SCAN calculator use the `load_fp` function (for "load foundation potential") in matcalc.


In [ ]:
from matcalc import load_fp

calc = load_fp("TensorNet-MatPES-r2SCAN-v2025.1-PES")

Now we can attach the calculator to the `Atoms` object.


In [ ]:
atoms.calc = calc

## Static Calculation


We now have everything we need to run a calculation. Let's start with running a static (single-point) calculation.


In [ ]:
e_initial = atoms.get_potential_energy()
print(e_initial)

It is always important to understand the units of any given code. In the case of ASE, energies are given in eV (https://ase-lib.org/ase/units.html). There are 96.49 kJ/mol per eV.

An absolute energy is meaningless. It must be referenced to something. We will come back to this.


## Geometry Optimization


The structure we loaded may not be a minimum energy structure. We must run a geometry optimization to obtain this.

ASE comes with many optimization routines. We will use BFGS.


In [ ]:
from ase.optimize import BFGS

# Set up the BFGS optimizer and specify output file
opt = BFGS(atoms, trajectory="opt.traj")

In [ ]:
opt.run(fmax=0.01)  # Run until the maximum force is less than 0.01 eV/Å

The `Atoms` object is updated in place, meaning that all the properties of it are now for the optimized structure and not the initial one.


In [ ]:
e_final = atoms.get_potential_energy()  # get the optimized energy
print(e_final - e_initial)  # print the energy difference

In [ ]:
atoms.get_forces()  # N_atoms x 3 matrix to describe F_x, F_y, F_z

If want to find the $\max|\bold{F}|$ across the structure, we need to compute $|\bold{F}_i| = \sqrt{F_{x,i}^2 + F_{y,i}^2 + F_{z,i}^2}$ for each atom $i$.


In [ ]:
import numpy as np

# |F_i| for all atoms i
force_mags = np.linalg.norm(atoms.get_forces(), axis=1)  # axis=1 means row-wise norm

# max|F|
max_f = force_mags.max()
print(f"max|F| = {max_f}")

It looks like we found a lower energy structure. Let's take a look at the optimization trajectory now.


In [ ]:
trajectory = read("opt.traj", index=":")  # index=":" means read all the frames

In [ ]:
print(f"The trajectory has {len(trajectory)} frames")

In [ ]:
from ase.visualize import view

view(trajectory)

## Geometry Optimization (Positions + Unit Cell)


If you look closely, you will see that the atomic positions updated and the energy went down. The forces went down too.

But note that the unit cell shape and size did not change. In addition to the atomic positions, the unit cell can also be optimized. After all, if you made the unit cell too big, that may not be the lowest energy structure. We can optimize both the positions and unit cell in ASE simultaneously.

In ASE, this is done by wrapping the `Atoms` object in a "Cell Filter". Otherwise, the procedure remains largely the same.


In [ ]:
from ase.filters import FrechetCellFilter

# this tells ASE to optimize both atomic positions and cell parameters
fcf = FrechetCellFilter(atoms)


In [ ]:
from ase.optimize import BFGS

opt = BFGS(fcf, trajectory="opt2.traj")
opt.run(fmax=0.01)

In [ ]:
atoms.get_potential_energy()  # get the optimized energy

In [ ]:
view(read("opt2.traj", index=":"))

In [ ]:
a, b, c = atoms.cell.lengths()
alpha, beta, gamma = atoms.cell.angles()

In [ ]:
print(f"a = {a}, b = {b}, c = {c}, alpha = {alpha}, beta = {beta}, gamma = {gamma}")

The experimental values for the primitive cell are: a = b = c = 18.303 angstroms and alpha = beta = gamma = 30.0 degrees. Not bad!

For reference, the values for the conventional cell are a = b = c = 25.885 angstroms and alpha = beta = gamma = 90.0 degrees.


## Using Other Calculators


Finally, we will note that there are many ASE calculators, some based on empirical potentials, some based on ML models, some based on quantum chemistry, and so on. An incomplete list can be found at https://ase-lib.org/ase/calculators/calculators.html. It is easy to swap to a different calculator and to do the same procedure as above.

For instance, we could run a calculation with a Lennard-Jones potential in exactly the same way. Note: Of course, the LJ potential we have chosen here is completely unphysical.


In [ ]:
from ase.calculators.lj import LennardJones

In [ ]:
# We will run a toy single-point calculation with an LJ potential
# This is completely unphysical because we have arbitrarily chosen sigma=1.0 and epsilon=1.0
# But it serves to illustrate how to set up a different calculator and also how
# to specify keyword arguments

atoms.calc = LennardJones(sigma=1.0, epsilon=1.0)
atoms.get_potential_energy()